In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://pbs.twimg.com/media/GG3SidNWYAAljb1?format=jpg&name=4096x4096)

# To Know about Google Gemini Architecture and Parameters, Watch this Video
## Link - https://youtu.be/vabQP3UkWDc

### Key Features of Gemini
- Enhanced Contextual Understanding: Gemini is the first model to outperform human experts on MMLU (Massive Multitask Language Understanding), one of the most popular methods to test the knowledge and problem-solving abilities of AI models.
- Multimodality: Gemini is built from the ground up for multimodality — reasoning seamlessly across text, images, video, audio, and code.
- Anything to anything: Gemini is natively multimodal, which gives you the potential to transform any type of input into any type of output.
- Customizability: Users can fine-tune Gemini for specific tasks or industries.


# NoteBook - Google Gemini Gemma Models Tutorials - 1

### Exmaple Code to Run Google Gemma Models

In [15]:
# Setup the environment
!pip install --upgrade huggingface_hub

  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install git+https://github.com/huggingface/transformers -U
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
!pip install trl
!pip install git+https://github.com/huggingface/peft.git

In [6]:
from huggingface_hub import notebook_login
notebook_login(write_permission=True)

The gemma models will be loaded as a pytorch model, so we can use torch to specify datatypes and other aspects of our model when we load it in. By default they will be loaded as torch.float32, meaning each weight will be made up of 32 bits of information, for our 7b model this means it will take about 32gb of memory needed to load this model

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM

"""Gemma 2b and 7b has a context window of 8192 tokens


"""
model_checkpoint = "google/gemma-2b-it"

# Load the model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForCausalLM.from_pretrained(model_checkpoint, device_map="cpu")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
print(model)
print(model.dtype)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaR

In [17]:
prompt = "7 Wonders of World?"

token_inputs = tokenizer(prompt, return_tensors="pt")

token_outputs = model.generate(input_ids=token_inputs['input_ids'], max_new_tokens=500)
# since this is exploratory we'll decode special tokens as well. for this prompt
new_tokens = token_outputs[0][token_inputs['input_ids'].shape[-1]:]
# it will be the beginning <bos> token and the ending <eos> tokens.
decoded_output = tokenizer.decode(new_tokens, skip_special_tokens=False)
decoded_output

"\n\nSure, here's a list of the 77 Wonders of the World:\n\n1. Great Pyramid of Giza\n2. The Great Sphinx\n3. Petra's Treasury\n4. The Hanging Gardens of Babylon\n5. The Great Wall of China\n6. The Colosseum in Rome\n7. The Statue of Zeus at Olympia\n8. The Hanging Gardens of Babylon\n9. The Great Pyramid of Giza\n10. The Valley of the Kings in Egypt\n11. The Temple of Artemis at Ephesus\n12. The Hanging Gardens of Babylon\n13. The Great Sphinx\n14. The Valley of the Kings in Egypt\n15. The Statue of Zeus at Olympia\n16. The Hanging Gardens of Babylon\n17. The Great Pyramid of Giza\n18. The Valley of the Kings in Egypt\n19. The Temple of Artemis at Ephesus\n20. The Hanging Gardens of Babylon\n21. The Great Pyramid of Giza\n22. The Valley of the Kings in Egypt\n23. The Temple of Artemis at Ephesus\n24. The Hanging Gardens of Babylon\n25. The Great Pyramid of Giza\n26. The Valley of the Kings in Egypt\n27. The Temple of Artemis at Ephesus\n28. The Hanging Gardens of Babylon\n29. The Grea

In [ ]:
print(decoded_output)

### Fine Tuning with LORA

In [ ]:
!pip install -U datasets
!pip install trl
!pip install git+https://github.com/huggingface/peft.git
# Restar the Notebook After this and Then Load Tokenizer and Model and Then Load the Dataset

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

# Load the pre-trained model and tokenizer
model_id = "google/gemma-2b"
lora_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=lora_config)

# Define PEFT configuration
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=4,
    lora_alpha=16,
    lora_dropout=0.01,
)

# Attach trainable adapters to the quantized model
peft_model = get_peft_model(model, peft_config)

# Load the dataset
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

import transformers
from trl import SFTTrainer

def formatting_func(example):
    text = f"Quote: {data['train']['quote'][0]}\nAuthor: {data['train']['author'][0]}"
    return [text]

2024-03-20 12:34:57.466464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 12:34:57.466528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 12:34:57.468303: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transf

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./fine-tuned_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,  # Reduce batch size
    gradient_accumulation_steps=4,  # Accumulate gradients
    learning_rate=2e-4,
    fp16=True,  # Enable mixed-precision training
    logging_steps=1,
    optim="paged_adamw_8bit"
)

# Create SFTTrainer
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data["train"],
    args=training_args,
    peft_config=peft_config,
    formatting_func=formatting_func,
)

# Fine-tune the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
wandb: Currently logged in as: simranjeetsingh1497 (simranjeet97). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1,1.032200
2,1.003200
3,0.982800


TrainOutput(global_step=3, training_loss=1.0060708324114482, metrics={'train_runtime': 40.1608, 'train_samples_per_second': 0.224, 'train_steps_per_second': 0.075, 'total_flos': 1926839549952.0, 'train_loss': 1.0060708324114482, 'epoch': 3.0})

In [4]:
peft_model.save_pretrained("gemma_2b_quotes_Fine_tuned")
tokenizer.save_pretrained("gemma_2b_quotes_Fine_tuned_tokenizer")

('gemma_2b_quotes_Fine_tuned_tokenizer/tokenizer_config.json',
 'gemma_2b_quotes_Fine_tuned_tokenizer/special_tokens_map.json',
 'gemma_2b_quotes_Fine_tuned_tokenizer/tokenizer.model',
 'gemma_2b_quotes_Fine_tuned_tokenizer/added_tokens.json',
 'gemma_2b_quotes_Fine_tuned_tokenizer/tokenizer.json')

In [5]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/gemma_2b_quotes_Fine_tuned_tokenizer")
quantized_model = AutoModelForCausalLM.from_pretrained("/kaggle/working/gemma_2b_quotes_Fine_tuned")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
text = "Quote: Life is"
inputs = tokenizer(text, return_tensors="pt")

outputs = quantized_model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Life is a journey, not a destination.

I am a wife, mother, and a professional photographer. I am a lover of all things creative, and I am a dreamer. I am a dreamer of dreams, and I am a dreamer of possibilities.


In [12]:
total_params = sum(p.numel() for p in quantized_model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 2506633216
